# Исследование надежности заемщиков

**Цель проекта:**

На основе статистики о платёжеспособности клиентов исследовать влияет ли семейное положение и количество детей клиента на факт возврата кредита в срок

**Описание проетка**

На основе данных кредитного отдела банка исследовал влияние семейного положения и
количества детей на факт погашения кредита в срок. Была получена информация о
данных. Определены и обработаны пропуски. Заменены типы данных на соответствующие
хранящимся данным. Удалены дубликаты. Категоризованы данные. Один датафрейм декомпозирован на три.

**Данное исследование разделим на несколько частей:**

* [ Откройте файл с данными и изучите общую информацию.](#1-section)
* [Предобработка данных.](#2-section)
* [Проверка гипотез.](#3-section)
* [Общий вывод по проекту.](#4-section)

<a id='1-section'></a>
# Открываем файл с данными и изучаем общую информацию

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('C:/Users/Гоша/OneDrive/Рабочий стол/DS/datasets/data.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Изучим общую информацию**

Расшифровка столбцов

- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность /по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

In [4]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [5]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [6]:
df.duplicated().sum()

54

- Столбец с трудовым стажем имеет тип данных `float`, необходимо изменить на `int`
- В данных есть дубликаты, которые нужно обработать
- В двух столбцах есть одинаковые количество пропусков, такие данные необходимо проверить
- Кол-во отработанных дней имеют отрицательные значения, но на первый взгляд выглядят выглядят адекватными

In [7]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


    - Отрицательное количество детей выглядит говорит нам о возможной ошибке в заполнении данных. Также максимальное кол-во детей(20) сильно выбивается из среднего значения.
    - Среднее количество отработанных дней больше 170 лет, а максимальное более 1100 лет (попробовать найти соотношение ошибок с другими данными)
    - Минимальный возраст 0 лет (колонка не важна для поставленных задач)

**Вывод**

В колонке с количеством детей отрицательное значение можно удалить и  20 детей это тоже опечатка, которую можно удалить. В колонке с трудовым стажем достаточно много непонятного: во-первых разобраться с отрицательными значениями, но вопрос с рекордсменами трудящимися более 100 лет остается, попробую сопоставить с возрастом, а возможно это всего несколько ошибок дают смещение.

#  Предобработка данных <a id='2-section'></a>

Проверим совпадают ли строки с пропусками

In [8]:
df.loc[df['days_employed'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


Так как строки совпадают и они занимают окло 10% от всех данных нам необходимо их обработать. Наилучшим выходом в этой ситуации будет заполнение пропусков медианным значением в соотвествии с типом занятости клиентов банка. Перед этим обработаем аномальные(отрицательные) значение в столбце общий трудовой стаж('days_employed').

In [9]:
df['days_employed'] = df['days_employed'].abs()

In [10]:
for t in df['income_type'].unique():
    df.loc[(df['income_type'] == t) & (df['total_income'].isna()), 'total_income'] = \
    df.loc[(df['income_type'] == t), 'total_income'].median()

In [11]:
df.groupby('income_type')['days_employed'].median().astype('int')

income_type
безработный        366413
в декрете            3296
госслужащий          2689
компаньон            1547
пенсионер          365213
предприниматель       520
сотрудник            1574
студент               578
Name: days_employed, dtype: int32

In [12]:
df = df[(df['children'] != -1) & (df['children'] != 20)]

In [13]:
df['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

Заполним пропуски в столбце `days_employed` медианными значениями по каждому типу занятости `income_type`.

In [14]:
for i in df['income_type'].unique():
    df.loc[(df['income_type'] == i) & (df['days_employed'].isna()), 'days_employed'] = \
    df.loc[(df['income_type'] == i), 'days_employed'].median()

Убедимся, что все пропуски заполнени

In [15]:
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [16]:
df['total_income'] = df['total_income'].astype('int')

Посмотрим есть ли неявные дубликаты.

In [17]:
df[df.duplicated()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,1572.328285,41,среднее,1,женат / замужем,0,F,сотрудник,0,142594,покупка жилья для семьи
4182,1,1572.328285,34,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,142594,свадьба
4851,0,365286.622650,60,среднее,1,гражданский брак,1,F,пенсионер,0,118514,свадьба
5557,0,365286.622650,58,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу
7808,0,365286.622650,57,среднее,1,гражданский брак,1,F,пенсионер,0,118514,на проведение свадьбы
8583,0,365286.622650,58,высшее,0,Не женат / не замужем,4,F,пенсионер,0,118514,дополнительное образование
9238,2,1572.328285,34,среднее,1,женат / замужем,0,F,сотрудник,0,142594,покупка жилья для сдачи
9528,0,365286.622650,66,среднее,1,вдовец / вдова,2,F,пенсионер,0,118514,операции со своей недвижимостью
9627,0,365286.622650,56,среднее,1,женат / замужем,0,F,пенсионер,0,118514,операции со своей недвижимостью
10462,0,365286.622650,62,среднее,1,женат / замужем,0,F,пенсионер,0,118514,покупка коммерческой недвижимости


Приведем столбец `education` к нижнему регистру.

In [18]:
df['education'] = df['education'].str.lower()

Удалим явные дубликаты.

In [19]:
df = df.drop_duplicates()

**Выделим столбец с категориями**

Проведем катекоризацию в соответсвие с ежемесечным доходом клиентов банка:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [20]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [21]:
df['total_income_category'] = df['total_income'].apply(categorize_income)

Выделим категории по целям оформления кредита.

In [22]:
df['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Исходя из выше представленных данных можно выделить 4 основные категории:
- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

In [23]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [24]:
df['purpose_category'] = df['purpose'].apply(categorize_purpose)

Проверим получившиеся данные

In [25]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


# Проверка гипотез:<a id='3-section'></a>

 - Есть ли зависимость между семейным положением и возвратом кредита в срок?
 - Есть ли зависимость между наличием детей и возвратом кредита в срок?
 - Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
 - Как разные цели кредита влияют на его возврат в срок?

**Есть ли зависимость между семейным положением и возвратом кредита в срок?**

In [26]:
df_status_pivot = df.pivot_table(
    'debt', 
    index=['family_status'], 
    aggfunc=['count','sum',lambda x: '{:.2%}'.format(x[:].mean())],
)
df_status_pivot = df_status_pivot.rename(
    columns={'sum':'Кол-во должников',
            'count':'Кол-во заемщиков',
            '<lambda>':'Доля должников', 'debt': ''}
)
df_status_pivot.index.name = 'Семейный статус'
display(df_status_pivot)

,Кол-во заемщиков,Кол-во должников,Доля должников
,,,
Семейный статус,,,
Не женат / не замужем,2796,273,9.76%
в разводе,1189,84,7.06%
вдовец / вдова,951,63,6.62%
гражданский брак,4134,385,9.31%
женат / замужем,12261,927,7.56%


**Вывод:**
Cемьи чаще пользуются услугами кредитования и имеют наибольшую сумму задолжности. Значительное среднее значение задолжности по кредиту имееют не женатые/не замужные люди, это связанно с тем, что уровень дохода семей выше.

**Есть ли зависимость между наличием детей и возвратом кредита в срок?**

In [27]:
df_children_pivot = df.pivot_table(
    'debt',
    index='children',
    aggfunc=['sum', 'count', lambda x: '{:.2%}'.format(x[:].mean())]
)
df_children_pivot =df_children_pivot.rename(
    columns={'sum':'Кол-во должников',
            'count':'Кол-во заемщиков',
            '<lambda>':'Доля должников', 'debt': ''}
)
df_children_pivot.index.name = 'Кол-во детей'
display(df_children_pivot)

,Кол-во должников,Кол-во заемщиков,Доля должников
,,,
Кол-во детей,,,
0,1063,14091,7.54%
1,444,4808,9.23%
2,194,2052,9.45%
3,27,330,8.18%
4,4,41,9.76%
5,0,9,0.00%


**Вывод:** чаще всего услугами банками по кредитованию пользуются люди не имеющие детей и имеют наименьшую среднюю задолжность. Это связано с отсутствием расходов на детей. С увеличением количества детей средняя задолжность имеет тенденцию к расту. Из общей тенденции выделяются семьи с 5-тью детьми, но для изучения данной группы кол-во данных не хватает.

**Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

In [28]:
df_income_pivot = df.pivot_table(
    'debt', 
    index = ['total_income_category'], 
    aggfunc = ['count','sum', lambda x: '{:.2%}'.format(x[:].mean())]
)
df_income_pivot = df_income_pivot.rename(
    columns={'sum':'кол-во должниокв',
            'count': 'кол-во заемщиков',
            '<lambda>' : 'доля должников', 'debt': ''}
)
df_income_pivot.index.name = 'Уровень дохода'
display(df_income_pivot)

,кол-во заемщиков,кол-во должниокв,доля должников
,,,
Уровень дохода,,,
A,25,2,8.00%
B,5014,354,7.06%
C,15921,1353,8.50%
D,349,21,6.02%
E,22,2,9.09%


**Вывод:** 
наибольшую среднюю задолжность по кредиту имеют люди с доходом до 30 тыс.руб., это связанно с довольно низким уровнем дохода. Можно сказать, что данна группа довольно безответственно относится к кредитному бремени. Наибольшую ответственность по возврату кредита проявляют люди с доходами 30-500 тыс.руб. и от 200 тыс.руб. до 1 миллиона рублей. Из 25 людей с доходами более 1 миллиона рублей двоя имеют задолжностью. Большинство из этой группы предприниматели и невозврат кредита в срок можно связать с определенными проблемами в бизнесе. Чаще всего услугами кредитования пользуются люди с доходом от 50-200тыс.руб. и сумма их задолжности имеет наибольшее значение.

**Как разные цели кредита влияют на его возврат в срок?**

In [29]:
df_category_pivot = df.pivot_table(
    'debt',
    index = ['purpose_category'],
    aggfunc = ['count','sum', lambda x: '{:.2%}'.format(x[:].mean())]
)
df_category_pivot = df_category_pivot.rename(
            {'sum':'кол-во должниокв',
            'count': 'кол-во заемщиков',
            '<lambda>' : 'доля должников', 'debt': ''}
)
df_category_pivot.index.name = 'Цели кредита'
display(df_category_pivot)

,count,sum,<lambda>
,debt,debt,debt
Цели кредита,,,
операции с автомобилем,4279,400,9.35%
операции с недвижимостью,10751,780,7.26%
получение образования,3988,369,9.25%
проведение свадьбы,2313,183,7.91%


**Вывод:**  самая популярная цель кредита это покупка жилья, средняя задолжность по ней самая минимальная, 99% людей закрывают кредит в срок. Самая большая средняя задолжность в 9,35% у людей, которые пользуются услугами банка с целью покупки автомобиля. На втором месте по невозврату кредита занимает получения образования 9,25%. Не у всех выпускников получается найти высокооплачиваемую работу для погашения кредита. 92% заемщиков, которые брали кредит с целью проведения свадьбы погошают его в срок.

# Общий вывод по проекту<a id='4-section'></a>

В результате проведенного исследования, целью которого было выявление различных фвкторов влияющих на возврат кредита в срок. Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

В исследовании были поставлены следущие гипотезы:

1.Влияет ли количество детей на погашение кредита в срок?
3.Какая зависимость между семейным положенем и возвратом кредита в срок?
4.Какая зависимость между уровнем дохода и возвратом кредита в срок?
5.Влияют ли цели кредита на возврат его в срок?
Гипотезы 1.

Изучив данные о количестве детей и погашением кредита в срок можно сделать следущие выводы: 92% заемщика, у которых нет детей, возвращают кредит в срок. С увеличением количества детей данный показатель уменьшается, данную тенденцию можно связать с увеличением расходов на детей. Таким образом 9,2% и 9,4% заемщиков не возвращают кредит в срок на них приходится по 1 и 2 ребенка соответственно.

Для подтверждения данной гипотезы необходимо провести дополнительное исследование на большей выборке. Так как со временем возможно увеличение количества детей.

Гипотезы 2.

Изучив данные о семейном положении и погашением кредита в срок можно сделать следущие выводы: 9,7% заемщиков не находящиеся в браке и 9,3% заемщиков в гражданском браке не возвращают кредит в срок. У этой группы больше шансов стать должниками. Люди находящиеся в разводе(7%), вдова/вдовец(6,6%) и женат/замужем(7,5%) имеют меньше шансов просрочить кредит. Данную гипотезу можно использовать.

Гипотезы 3.

Изучив данные о уровнем дохода и погашением кредита в срок можно сделать следущие выводы: 8,5% доля просроченных кредитов у группы людей с доходами 50-200тыс.руб. 7% доля просроченных кредитов у группы людей с доходами 200-1000тыс.руб. Для подтверждение данной гипотезы необходимо дополнительное исследование с большим количеством данных. Гипотезы 4.

Изучив данные на какие цели берутся кредиты и погашением кредита в срок можно сделать следущие выводы: 7,2% доля просроченных кредитов, целью которых было операции с недвижимостью. 7,7% доля просроченных кредитов, целью которых проведение свадьбы. 9,2% доля просроченных кредитов, целью которых получение образования. 9,3% доля просроченных кредитов, целью которых операции с автомобилем. Данная гипотеза подверждена и её можно использовать.

Таким образом специалистам, которые занимаются скорингом можно использовать 2 и 4 гипотезу.